# Full Test-Ledoit2017 Table 1

In [46]:
import numpy as np
import pandas as pd
import nonlinshrink as nls

def showmatrixinfo(matrix):
    print("Matrix shape: ",np.shape(matrix))
    print(matrix)

## Data import

In [44]:
# stocksdata=stocks_u1.iloc[i*21:271+i*21,1:101].to_numpy()
# yt_est=stocksdata[:250,].T
# yt_test=stocksdata[250:,].T
# showmatrixinfo(yt_est)
# # showmatrixinfo(yt_test)

In [29]:
# #Load and check data

# yt_est=np.load("Fix100Stock_est_matrix.npy")
# yt_test=np.load("Fix100Stock_test_matrix.npy")

# FFdata = pd.read_csv('F-F_Research_Data_Factors_daily_Fix100.csv')
# index=["Mkt-RF","SMB","HML"]
# FFfactors=FFdata.loc[:249,index].to_numpy()
# rf_rate=FFdata.loc[250:,"RF"].to_numpy()
# rf_daily=rf_rate/250                          


# print(np.shape(yt_est))               
# print(np.shape(yt_test)) 
# print(np.shape(rf_daily))
# print(yt_test)
# data=[yt_est,yt_test,FFdata]

# index=["Mkt-RF","SMB","HML"]
# FFfactors=FFdata.loc[:249,index].to_numpy()

## Empirical test

### Functions & data

In [47]:
## Functions:

def test_result(omega,method):
    ### Use estimated covariance matrix to calculate w
    Omega= np.matrix(omega)
    w=(Omega.I@np.ones((100,1)))/(np.ones((1,100))@Omega.I@np.ones((100,1)))
    print("\n"+method+":")
    print("weight_vector:")
    print(np.shape(w))
    
    ###########calculate payoff and excess payoff(ri-rf)
    
    payoff=w.T@yt_test*100
    
    exc_payoff=payoff-rf_daily
    print("Return: "+method)
    print(type(exc_payoff.tolist()[0]))
    print("lenth:")
    print(len(exc_payoff.tolist()[0]))
    
    
    return exc_payoff.tolist()[0]

### Methods:

In [50]:
def run_1N():
    N=100
    omega=np.identity(100)
    
    return1N=test_result(omega,"1/N")
    return return1N

def run_Samp():
    ## ---------Sample Covariance Estimation
    omega=np.cov(yt_est)
    print("Matrix shape:")
    print(np.shape(omega))

    returnSamp=test_result(omega,"Samp")
    return returnSamp
    
def run_Lin():
    ##--------Lin(Ledoit-Wolf 2004 methpd) shrinkage Estimation
    from sklearn.covariance import ledoit_wolf

    omega=ledoit_wolf(yt_est.T)[0]

    returnLin=test_result(omega,"Lin")
    return returnLin

##-------NonLin shrinkage Estimation--------------

def run_NonLin():
    import nonlinshrink as nls

    omega=nls.shrink_cov(yt_est.T)
    print(np.shape(omega))

    returnNonLin=test_result(omega,"NonLin")
    return returnNonLin

##--------Single factor estimation---------------------##

def run_SF():
        ##   Generate Factor
    equalw=np.array([[0.01]*100])
    print("equal portfolio: ",np.shape(equalw))
    factor=equalw@yt_est
    print("Factor matrix shape: ",np.shape(factor))


    ##estimate SigmaF
    var_f=np.var(factor,ddof=1)      #variance of factor,

    np.savetxt("SF_factor.csv",factor,delimiter=",")

    ##compute cov(Ri,Rf),the covariance of stocks and factor
    var_if=np.cov(yt_est,factor)[-1,:-1]
    var_if=np.matrix(var_if)          #convert to 1X100 matrix


    SigmaSF=var_if.T*var_if/var_f  
    for i in range(100):
        SigmaSF[i,i]=np.cov(yt_est)[i,i]


    showmatrixinfo(SigmaSF)    ####Seems correct

    returnSF=test_result(SigmaSF,"SF")
    return returnSF,SigmaSF


##-----------------FAMA FRENCH estimation------------------------

def run_FF():
    #####First, Generate 3-factors array.


    from sklearn.linear_model import LinearRegression

    LG=LinearRegression()

    LG.fit(FFfactors,yt_est.T)      ##FFfactor matirx is a (250,3) matrix!
    betas=LG.coef_
    print("Beta matrix: ",np.shape(betas))

    var_ff=np.cov(FFfactors.T)      ##Covariance of FAMA FRENCH 3 Factor model.

    SigmaF=betas@var_ff@betas.T

    ###As same as SF, the diagonal need add residual,or replace by var(Ri)

    for i in range(100):
        SigmaF[i,i]=np.cov(yt_est)[i,i]

    print("SigmaF: ")
    showmatrixinfo(SigmaF)

    returnFF=test_result(SigmaF,"FF")
    return returnFF



def run_POET():

#---------POET estimation-----------------#
    from sklearn.decomposition import PCA
    
    print("generating components:")
    pca = PCA(n_components=5, copy=True)
    pca.fit(yt_est)
    factors=pca.components_
    print('PCA Componets:\n', pca.components_)
    print(np.shape(pca.components_))
    print('\nEigenvalues:', pca.explained_variance_)
    print('Variance explaination:', pca.explained_variance_ratio_)
    print("Add up: ",np.sum(pca.explained_variance_ratio_))


    ##Regression:
    from sklearn.linear_model import LinearRegression

    print("\nRegression:")
    LG2=LinearRegression()

    LG2.fit(factors.T,yt_est.T)
    betas=LG2.coef_
    print("Beta matrix: ",np.shape(betas))

    var_fs=np.cov(factors)

    SigmaF=betas@var_fs@betas.T

    ###As same as SF, the diagonal need add residual,or replace by var(Ri)

    for i in range(100):
        SigmaF[i,i]=np.cov(yt_est)[i,i]

    showmatrixinfo(SigmaF)

    returnPOET=test_result(SigmaF,"POET")
    return returnPOET


def run_NLSF(SigmaSF):
    ## NL-SF
    eigenvalue, eigenvectors = np.linalg.eig(SigmaSF)
    print(np.shape(eigenvalue))
    print(np.shape(eigenvectors))

    print(np.allclose(SigmaSF,eigenvectors@np.diag(eigenvalue)@eigenvectors.T))

    diag=np.identity(100)
    diag2=np.zeros((100,100))
    for i in range(100):
        diag[i,i]=pow(eigenvalue[i],-1/2)
        diag2[i,i]=pow(eigenvalue[i],1/2)
    ##Generate Yt x Sigma_SF to the power of -1/2
    SigmaSF2=eigenvectors@diag@eigenvectors.T  ##(1/2)
    SigmaSF3=eigenvectors@diag2@eigenvectors.T  ##(-1/2)
    SigmaC_hat=nls.shrink_cov(yt_est.T@SigmaSF2)

    #Reincorporating the structure.
    SigmaNLSF=SigmaSF3@SigmaC_hat@SigmaSF3
    returnNLSF=test_result(SigmaNLSF,"NL-SF")
    return returnNLSF


### Running test

In [52]:
#----------------Initializing-------------#
return1N=[]
returnSamp=[]
returnLin=[]
returnNonLin=[]
returnSF=[]
returnFF=[]
returnPOET=[]
returnNLSF=[]

def run(data):
    yt_est=data[0]
    yt_test=data[1]    
    FFdata=data[2]

    return1N.extend(run_1N())
    returnSamp.extend(run_Samp())
    returnLin.extend(run_Lin())
    returnNonLin.extend(run_NonLin())
    returnSF.extend(run_SF()[0])
    returnFF.extend(run_FF())
    returnPOET.extend(run_POET())
    returnNLSF.extend(run_NLSF(run_SF()[1]))
    
    return


#####--------------Running test--------------
FF_u1 = pd.read_csv('data/FF_universe1.csv')
stocks_u1=pd.read_csv('data/stocks100_u1.csv')

FF_u2= pd.read_csv('data/FF_universe2.csv')
stocks_u2=pd.read_csv('data/stocks100_u2.csv')

def test_result(omega,method):
    ### Use estimated covariance matrix to calculate w
    Omega= np.matrix(omega)
    w=(Omega.I@np.ones((100,1)))/(np.ones((1,100))@Omega.I@np.ones((100,1)))
    print("\n"+method+":")
    print("weight_vector:")
    print(np.shape(w))
    
    ###########calculate payoff and excess payoff(ri-rf)
    
    payoff=w.T@yt_test*100
    
    exc_payoff=payoff-rf_daily
    print("Return: "+method)
    print(type(exc_payoff.tolist()[0]))
    print("lenth:")
    print(len(exc_payoff.tolist()[0]))
    
    
    return exc_payoff.tolist()[0]


def iteration(FF_u,stocks_u):
    
    for i in range(2): 
        FFdata = FF_u.iloc[i*21:271+i*21,]

        stocksdata=stocks_u.iloc[i*21:271+i*21,1:101].to_numpy()
        yt_est=stocksdata[:250,].T
        yt_test=stocksdata[250:,].T

        index=["Mkt-RF","SMB","HML"]
        FFfactors=FFdata.loc[:,index].iloc[:250,].to_numpy()
        rf_rate=FFdata.loc[:,"RF"].iloc[250:,].to_numpy()
        rf_daily=rf_rate/250                          

        data=[yt_est,yt_test,FFdata]
        print(data[1])
        run(data)
        
        print("Tetst "+str(i+1)+" complete!")
    return

#--------------main-----------------#

iteration(FF_u1,stocks_u1)

print("Universe 1 Complete!")




    


ret_mat=np.array([return1N,returnSamp,returnLin,returnNonLin,returnSF,returnFF,returnPOET,returnNLSF])

print("Return maxtrix:")
print(np.shape(ret_mat))


[[-0.002959  0.020772  0.017442 ... -0.002857  0.        0.      ]
 [-0.004717  0.009479 -0.002347 ...  0.004717 -0.002347 -0.002353]
 [-0.021858 -0.005587  0.011236 ...  0.005464 -0.032609  0.016854]
 ...
 [ 0.026022 -0.01087   0.005495 ...  0.012448 -0.002049  0.00616 ]
 [ 0.        0.       -0.05625  ... -0.007299  0.007353  0.014599]
 [ 0.046414  0.040323  0.015504 ... -0.00463  -0.009302  0.004695]]

1/N:
weight_vector:
(100, 1)


NameError: name 'rf_daily' is not defined

### Ploting results

In [40]:
#####Show all the results

AV=np.mean(ret_mat,axis=1)*250
AV=AV.T.tolist()
    
SD=np.std(ret_mat,axis=1)*pow(250,.5)
SD=SD.T.tolist()

SR=[]
for i in range(len(SD)):
    SR.append(AV[i]/SD[i])





def prtb():
    print("Table 1")
    print("Performance measures for various estimators of the GMV portfolio")
    print("{}".format('''Period: January 19, 1973 to December 31, 2011" '''))
    print("\t1/N \tSample \tLin \tNolin \tSF \tFF \tPOET \tNL-SF")
    print("{:-^70}".format(""))
    print("{:^70}".format("N=100"))
    print("{:-^70}".format(""))
    print("AV \t{:.2f} \t{:.2f}\t{:.2f}\t{:.2f}\t{:.2f}\t{:.2f}\t{:.2f}\t{:.2f}".format(
        AV[0],AV[1],AV[2],AV[3],AV[4],AV[5],AV[6],AV[7])
    )
    print("SD \t{:.2f} \t{:.2f}\t{:.2f}\t{:.2f}\t{:.1f}\t{:.2f}\t{:.2f}\t{:.2f}".format(
        SD[0],SD[1],SD[2],SD[3],SD[4],SD[5],SD[6],SD[7]))
    print("SR \t{:.2f} \t{:.2f}\t{:.2f}\t{:.2f}\t{:.2f}\t{:.2f}\t{:.2f}\t{:.2f}".format(
        SR[0],SR[1],SR[2],SR[3],SR[4],SR[5],SR[6],SR[7]))
        
    return


prtb()

Table 1
Performance measures for various estimators of the GMV portfolio
Period: January 19, 1973 to December 31, 2011" 
	1/N 	Sample 	Lin 	Nolin 	SF 	FF 	POET 	NL-SF
----------------------------------------------------------------------
                                N=100                                 
----------------------------------------------------------------------
AV 	nan 	nan	nan	nan	nan	nan	nan	nan
SD 	nan 	nan	nan	nan	nan	nan	nan	nan
SR 	nan 	nan	nan	nan	nan	nan	nan	nan


/opt/network/anaconda3/lib/python3.9/site-packages/numpy/core/fromnumeric.py:3432: RuntimeWarning: Mean of empty slice.
  return _methods._mean(a, axis=axis, dtype=dtype,
/opt/network/anaconda3/lib/python3.9/site-packages/numpy/core/_methods.py:182: RuntimeWarning: invalid value encountered in divide
  ret = um.true_divide(
/opt/network/anaconda3/lib/python3.9/site-packages/numpy/core/_methods.py:265: RuntimeWarning: Degrees of freedom <= 0 for slice
  ret = _var(a, axis=axis, dtype=dtype, out=out, ddof=ddof,
/opt/network/anaconda3/lib/python3.9/site-packages/numpy/core/_methods.py:223: RuntimeWarning: invalid value encountered in divide
  arrmean = um.true_divide(arrmean, div, out=arrmean, casting='unsafe',
/opt/network/anaconda3/lib/python3.9/site-packages/numpy/core/_methods.py:254: RuntimeWarning: invalid value encountered in divide
  ret = um.true_divide(
